# HOG Computer Vision Homework - Analysis Notebook

Bu notebook, projenin analizi ve rapor için grafik oluşturma amacıyla kullanılır.

## Proje Özeti

Bu ödev üç ana problemden oluşmaktadır:

1. **Problem 1: HOG Özellik Çıkarımı** - HOG algoritmasının sıfırdan implementasyonu
2. **Problem 2: Nesne Tespiti** - İnsan tespiti ve özel nesne tespiti (HOG + SVM)
3. **Problem 3: Sınıflandırma** - HOG özellikleri ile görüntü sınıflandırma

## Kullanım

Bu notebook'u şunlar için kullanın:
- Farklı HOG parametreleri ile deneyler yapma
- HOG özelliklerini görselleştirme
- Tespit sonuçlarını karşılaştırma
- Rapor için grafikler oluşturma (bar chart, confusion matrix, vb.)
- Sınıflandırma performansını analiz etme



In [ ]:
# Gerekli kütüphaneleri import et
import sys
import os
from pathlib import Path

# Proje root dizinini path'e ekle
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / "src"))

import numpy as np
import matplotlib.pyplot as plt
import cv2
import joblib

# Proje modüllerini import et
from src.hog_implementation import (
    compute_gradients,
    compute_hog_descriptor,
    visualize_hog,
    save_gradient_outputs
)
from src.object_detection import (
    detect_humans_in_image,
    run_human_detection_dataset,
    train_custom_svm_classifier,
    detect_custom_objects_with_svm,
    run_custom_detection_dataset_with_svm
)
from src.classification import (
    load_dataset_from_folder,
    extract_features_hog_skimage,
    extract_features_hog_custom,
    train_svm_classifier,
    evaluate_classifier,
    plot_confusion_matrix,
    run_classification_experiment,
    compare_hog_implementations
)
from src.utils import load_grayscale_image, load_images_from_folder

print("✓ Tüm import'lar başarılı!")
print(f"✓ Proje root: {project_root}")


## Problem 1: HOG Özellik Çıkarımı Deneyleri

Bu bölümde HOG implementasyonunuzu test edebilir ve farklı parametrelerle deneyler yapabilirsiniz.

- Test görüntülerini yükleme
- Farklı parametrelerle HOG descriptor hesaplama
- HOG özelliklerini görselleştirme
- Sonuçları karşılaştırma


In [ ]:
# Problem 1: HOG Özellik Çıkarımı Deneyleri

# Test görüntüsü yükle
image_path = "data/test_images/square.jpeg"
if os.path.exists(image_path):
    image = load_grayscale_image(image_path)
    print(f"✓ Görüntü yüklendi: {image_path}")
    print(f"  Görüntü boyutu: {image.shape}")
    
    # Gradient hesaplama
    Gx, Gy, magnitude, orientation = compute_gradients(image)
    print(f"✓ Gradientler hesaplandı")
    print(f"  Gx shape: {Gx.shape}, Gy shape: {Gy.shape}")
    
    # HOG descriptor hesaplama (farklı parametrelerle)
    print("\n--- HOG Descriptor Hesaplama ---")
    
    # Parametre seti 1: Standart (8x8 cell, 2x2 block, 9 bins)
    hog_desc_1 = compute_hog_descriptor(image, cell_size=(8, 8), block_size=(2, 2), num_bins=9)
    print(f"Parametre 1 (8x8, 2x2, 9): {hog_desc_1.shape} -> {hog_desc_1.size} özellik")
    
    # Parametre seti 2: Daha küçük cell (16x16)
    hog_desc_2 = compute_hog_descriptor(image, cell_size=(16, 16), block_size=(2, 2), num_bins=9)
    print(f"Parametre 2 (16x16, 2x2, 9): {hog_desc_2.shape} -> {hog_desc_2.size} özellik")
    
    # Parametre seti 3: Farklı bin sayısı
    hog_desc_3 = compute_hog_descriptor(image, cell_size=(8, 8), block_size=(2, 2), num_bins=6)
    print(f"Parametre 3 (8x8, 2x2, 6): {hog_desc_3.shape} -> {hog_desc_3.size} özellik")
    
    # HOG görselleştirme
    print("\n--- HOG Görselleştirme ---")
    hog_vis = visualize_hog(image, cell_size=(8, 8), num_bins=9, scale_factor=2.0)
    print(f"✓ HOG görselleştirmesi oluşturuldu: {hog_vis.shape}")
    
    # Gradient çıktılarını kaydet
    output_dir = "data/results/notebook_analysis"
    os.makedirs(output_dir, exist_ok=True)
    save_gradient_outputs(image_path, output_dir=output_dir)
    print(f"✓ Gradient çıktıları kaydedildi: {output_dir}")
    
else:
    print(f"⚠ Görüntü bulunamadı: {image_path}")
    print("Lütfen doğru dosya yolunu kontrol edin.")


## Problem 2: Nesne Tespiti Deneyleri

Bu bölümde nesne tespiti sistemlerini test edebilirsiniz:

- İnsan tespiti (OpenCV hazır model)
- Özel nesne tespiti (SVM ile eğitilmiş model)
- Tespit sonuçlarını karşılaştırma
- Performans analizi


In [ ]:
# Problem 2: Nesne Tespiti Deneyleri

# 2.1. İnsan Tespiti (Hazır Model)
print("=" * 60)
print("2.1. İNSAN TESPİTİ (OpenCV Hazır Model)")
print("=" * 60)

# Tek bir görüntüde insan tespiti
test_image_path = "data/test_images/000025.jpg"
if os.path.exists(test_image_path):
    image = cv2.imread(test_image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # İnsan tespiti yap
    rects, weights = detect_humans_in_image(gray)
    print(f"✓ Tespit tamamlandı: {len(rects)} insan bulundu")
    
    if len(rects) > 0:
        print(f"  Confidence skorları: {weights}")
    else:
        print("  ⚠ İnsan tespit edilmedi")
else:
    print(f"⚠ Test görüntüsü bulunamadı: {test_image_path}")

# Tüm test görüntülerinde insan tespiti
print("\n--- Tüm Test Görüntülerinde İnsan Tespiti ---")
print("Not: Bu işlem tüm data/test_images/ klasöründeki görüntüleri işler")
# run_human_detection_dataset()  # Yorumu kaldırarak çalıştırabilirsiniz

# 2.2. Özel Nesne Tespiti (SVM Modeli)
print("\n" + "=" * 60)
print("2.2. ÖZEL NESNE TESPİTİ (SVM Modeli)")
print("=" * 60)

# Model yolu kontrolü
model_path = "models/trained_classifier.pkl"
if os.path.exists(model_path):
    print(f"✓ Eğitilmiş model bulundu: {model_path}")
    classifier = joblib.load(model_path)
    print("✓ Model yüklendi")
    
    # Test görüntüsünde özel nesne tespiti
    test_image_path = "data/test_images/000001.jpg"
    if os.path.exists(test_image_path):
        image = cv2.imread(test_image_path)
        has_object, max_score, result_image = detect_custom_objects_with_svm(
            image, classifier, score_threshold=0.0
        )
        print(f"✓ Tespit tamamlandı")
        print(f"  Nesne var mı: {'EVET' if has_object else 'HAYIR'}")
        print(f"  Max score: {max_score:.3f}")
    else:
        print(f"⚠ Test görüntüsü bulunamadı: {test_image_path}")
else:
    print(f"⚠ Eğitilmiş model bulunamadı: {model_path}")
    print("  Model eğitmek için: train_custom_svm_classifier() fonksiyonunu kullanın")
    # classifier = train_custom_svm_classifier()  # Yorumu kaldırarak model eğitebilirsiniz


In [ ]:
# Problem 3: Sınıflandırma Deneyleri

base_folder = "data/training_set"

if os.path.exists(base_folder):
    print("=" * 60)
    print("3.1. scikit-image HOG ile Sınıflandırma")
    print("=" * 60)
    
    # scikit-image HOG ile sınıflandırma
    # run_classification_experiment(
    #     base_folder=base_folder,
    #     use_custom_hog=False,
    #     save_model_path="models/classification_skimage.pkl"
    # )
    print("Not: Yorumu kaldırarak deneyi çalıştırabilirsiniz")
    
    print("\n" + "=" * 60)
    print("3.2. Custom HOG ile Sınıflandırma")
    print("=" * 60)
    
    # Custom HOG ile sınıflandırma
    # run_classification_experiment(
    #     base_folder=base_folder,
    #     use_custom_hog=True,
    #     save_model_path="models/classification_custom.pkl"
    # )
    print("Not: Yorumu kaldırarak deneyi çalıştırabilirsiniz")
    
    print("\n" + "=" * 60)
    print("3.3. Her İki Implementasyonu Karşılaştırma")
    print("=" * 60)
    
    # Karşılaştırma
    # compare_hog_implementations(base_folder=base_folder)
    print("Not: Yorumu kaldırarak karşılaştırmayı çalıştırabilirsiniz")
    
else:
    print(f"⚠ Dataset klasörü bulunamadı: {base_folder}")
    print("Lütfen data/training_set/ klasörünü oluşturun ve içine negative/ ve positive/ alt klasörlerini ekleyin")


## Rapor Grafikleri

Bu bölümde rapor için grafikler oluşturabilirsiniz:

- Farklı HOG implementasyonları arasında accuracy karşılaştırması
- Confusion matrix görselleştirmeleri
- Tespit performans metrikleri
- Özellik görselleştirme grafikleri
- Dataset dağılım grafikleri


In [ ]:
# Rapor Grafikleri Oluşturma

# Çıktı klasörünü oluştur
figures_dir = "report/figures"
os.makedirs(figures_dir, exist_ok=True)

# 1. Accuracy Karşılaştırması (Bar Chart)
print("1. Accuracy Karşılaştırması Grafiği")
accuracies = {
    'scikit-image HOG': 0.9565,
    'Custom HOG': 0.9565
}

plt.figure(figsize=(10, 6))
bars = plt.bar(accuracies.keys(), accuracies.values(), color=['#3498db', '#2ecc71'])
plt.ylabel('Accuracy', fontsize=12)
plt.title('HOG Implementasyonu Karşılaştırması', fontsize=14, fontweight='bold')
plt.ylim([0, 1])
plt.grid(axis='y', alpha=0.3)

# Değerleri çubukların üzerine yaz
for bar, value in zip(bars, accuracies.values()):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{value:.4f}', ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.savefig(f'{figures_dir}/accuracy_comparison.png', dpi=300, bbox_inches='tight')
print(f"✓ Grafik kaydedildi: {figures_dir}/accuracy_comparison.png")
plt.show()

# 2. Dataset Dağılımı (Pie Chart)
print("\n2. Dataset Dağılım Grafiği")
base_folder = "data/training_set"
if os.path.exists(base_folder):
    negative_folder = os.path.join(base_folder, "negative")
    positive_folder = os.path.join(base_folder, "positive")
    
    negative_count = len([f for f in os.listdir(negative_folder) 
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))]) if os.path.exists(negative_folder) else 0
    positive_count = len([f for f in os.listdir(positive_folder) 
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))]) if os.path.exists(positive_folder) else 0
    
    if negative_count > 0 or positive_count > 0:
        labels = ['Negative', 'Positive']
        sizes = [negative_count, positive_count]
        colors = ['#e74c3c', '#27ae60']
        
        plt.figure(figsize=(8, 8))
        plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
        plt.title('Dataset Dağılımı', fontsize=14, fontweight='bold')
        plt.axis('equal')
        plt.tight_layout()
        plt.savefig(f'{figures_dir}/dataset_distribution.png', dpi=300, bbox_inches='tight')
        print(f"✓ Grafik kaydedildi: {figures_dir}/dataset_distribution.png")
        plt.show()
    else:
        print("⚠ Dataset klasörlerinde görüntü bulunamadı")
else:
    print(f"⚠ Dataset klasörü bulunamadı: {base_folder}")

# 3. Performans Metrikleri Karşılaştırması
print("\n3. Performans Metrikleri Grafiği")
metrics = {
    'Precision': [0.94, 1.00],
    'Recall': [1.00, 0.88],
    'F1-Score': [0.97, 0.93]
}
classes = ['Negative', 'Positive']
x = np.arange(len(classes))
width = 0.25

fig, ax = plt.subplots(figsize=(10, 6))
for i, (metric, values) in enumerate(metrics.items()):
    offset = (i - 1) * width
    bars = ax.bar(x + offset, values, width, label=metric)
    # Değerleri çubukların üzerine yaz
    for bar, value in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
               f'{value:.2f}', ha='center', va='bottom', fontsize=9)

ax.set_ylabel('Score', fontsize=12)
ax.set_title('Sınıf Bazlı Performans Metrikleri', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(classes)
ax.legend()
ax.set_ylim([0, 1.1])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(f'{figures_dir}/performance_metrics.png', dpi=300, bbox_inches='tight')
print(f"✓ Grafik kaydedildi: {figures_dir}/performance_metrics.png")
plt.show()

print("\n✓ Tüm grafikler oluşturuldu!")
print(f"Grafiklerin konumu: {figures_dir}/")
